In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import sys
import os

In [ ]:
DATATYPE_USED = "float32"
def cut_test_volume_data(path_in: str):
    with h5py.File(path_in, "r") as f_in:
        name_old = path_in.split(".hdf5")[0]
        name_new = name_old+"_test_5cut.hdf5"
        with h5py.File(name_new, "w") as f_out:
            new_volume_dataset = None

            # takeover all other existing groups beside "Volume"
            for key in f_in.keys():
                if key != "Volume":
                    # Get parent group name for copy
                    group_path = f_in[key].parent.name
                    # Check existence of group, else create group+parent
                    group_id = f_out.require_group(group_path)
                    f_in.copy(key, group_id, group_path+key)
            
            vol_hdf5 = f_in["Volume"]
            # Volume iteration loop
            grey_value_sum = 0
            y_slice_idx = np.linspace(0, vol_hdf5.shape[1]-1, 5, dtype=int)
            print("Take y slices:", y_slice_idx)
            for y_idx, y_slice in enumerate(y_slice_idx):
                vol_data = vol_hdf5[:, y_slice, :]
                grey_value_sum += vol_data.flatten().sum()

                vol_data_3d = np.expand_dims(vol_data, axis=0)
                if new_volume_dataset is None:
                    new_volume_dataset = f_out.create_dataset("Volume",
                        data=vol_data_3d, dtype=DATATYPE_USED, chunks=True,
                        maxshape=(None, None, None))
                else:
                    new_volume_dataset.resize(new_volume_dataset.shape[0] +
                        vol_data_3d.shape[0], axis=0)
                    new_volume_dataset[y_idx, :, :] = vol_data_3d
            
            new_volume_dataset.attrs['MATLAB_class'] = 'double'
            new_volume_dataset.attrs['Mean_grey_value'] = grey_value_sum/vol_hdf5.size

In [ ]:
path_in = "/net/pasnas01/pool1/enpro-2021-voxie/reconstructed_volumes/cable_holder_chain/poly/volume_transposed_and_cutted.hdf5"
cut_test_volume_data(path_in)

In [ ]:
def plot_slices_from_hdf5(path_in: str, clim):
    with h5py.File(path_in, "r") as f_in:
        # Volume iteration loop
        vol_hdf5 = f_in["Volume"]
        img_idx = np.linspace(0, vol_hdf5.shape[0]-1, 10, dtype=int)
        print("Take images:", img_idx)
        for y_idx, y_slice in enumerate(img_idx):
            vol_data = vol_hdf5[y_slice, :, :]
            img = np.array(vol_data, dtype=np.float32)
            print("Max:\t{} | Min:\t{}".format(np.amax(img), np.amin(img)))
            plt.figure()
            plt.imshow(img, cmap="gray")
            plt.clim(clim[0], clim[1])
            plt.colorbar()
            plt.title("Slice {}".format(y_slice))
            plt.savefig(str(path_in.split("/")[-1])+"img_"+str(y_slice)+".png")
        plt.show()
            

In [ ]:
#path_in = "/net/pasnas01/pool1/enpro-2021-voxie/reconstructed_volumes/cable_holder_chain/poly/volume.hdf5"
path_in = "/net/pasnas01/pool1/enpro-2021-voxie/reconstructed_volumes/cable_holder_chain/poly/volume_transposed_and_cutted.hdf5"
plot_slices_from_hdf5(path_in, [-500, 500])

In [ ]:
from dataloader import get_dataloader
from visualization import make_grid
import torch

def plot_256slices_from_hdf5(path_in: str, clim, loader, indices, rows):
    with h5py.File(path_in, "r") as f_in:
        # Volume iteration loop
        data = []
        cnt = 0
        for img, gt in iter(loader):
            if cnt % 5 == 0:
                print("Counter:", cnt)
            #data.append(img[:, 2, :, :])

            data.append(np.squeeze(np.array(img[:, 2, :, :])))
            plt.imshow(np.squeeze(np.array(img[:, 2, :, :])), cmap="gray")
            plt.clim(clim[0], clim[1])
            plt.colorbar()
            plt.show()
            cnt += 1
        data_stack = np.stack(data)
        print("Data stacked:", data_stack.shape)
        #print("Max:\t{} | Min:\t{}".format(np.amax(img), np.amin(img)))
        plt.figure(figsize=(12,8), dpi= 100)
        grid_array = make_grid(data_stack,rows)
        plt.imshow(grid_array, cmap="gray")
        plt.clim(clim[0], clim[1])
        plt.colorbar()

        #plt.savefig(str(path_in.split("/")[-1])+"img_"+str(y_slice)+".png")
            
        plt.show()
            

In [ ]:
from torch.utils.data.sampler import SequentialSampler
#path_in = "/net/pasnas01/pool1/enpro-2021-voxie/reconstructed_volumes/cable_holder_chain/poly/volume.hdf5"
path_in = "/home/so/TIFFs/vol_cut_cableHolder_mono.hdf5"

idx = np.arange(1000, 2000, 200)
print("Idx:", idx)
sampler = SequentialSampler(idx)
loader = get_dataloader(1, 2, 256, 128, [(path_in, path_in)], 
                                sampler=sampler, shuffle=False)
print("Loader data length:", len(loader.dataset))
print("Loader datasets:", loader.dataset.len_datasets)
print("Num datasets:", loader.dataset.num_datasets)

plot_256slices_from_hdf5(path_in, [-2, 100],loader, idx, 2)                    

In [2]:
import json

paths = [("ct_1", "gt_1"), ("ct_2", "gt_2"), ("ct_3", "gt_3")]
print(paths)

[('ct_1', 'gt_1'), ('ct_2', 'gt_2'), ('ct_3', 'gt_3')]
